# 2.0 Data Preprocessing 
## 2.1 Merging Raw Data

###### Author: Gan Yee Jing, Yeap Jie Shen
###### Last Edited: 31/08/2024

### 2.1.1 Importing Libraries 

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

import sys

sys.path.append(r'/home/student/RDS2S3G4_CLO2_B')

from data_stores.hdfsClient import HdfsClient
from data_stores.hbaseClient import HBaseClient
from data_stores.redisClient import RedisClient

import pickle

# Creating spark session
spark = SparkSession.builder.appName('merge raw data').getOrCreate()

24/09/02 08:38:46 WARN Utils: Your hostname, Gan. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/02 08:38:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/02 08:38:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/02 08:38:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### 2.1.3 Retrieving Raw Data from HDFS or Redis
If there is a cache in Redis, then the data is read from it
Otherwise, the data is read from HDFS

Reading data from Redis is faster as this is a in-memory data store

In [2]:
# Instantiating HDFS client
hdfs_client = HdfsClient(spark)

# Instantiating Redis client
redis_client = RedisClient(host = 'localhost', port = 6379, db = 0, start_now = True)

newsdata_io_schema = StructType([
            StructField('title', StringType(), True),
            StructField('article_id', StringType(), True),
            StructField('source_url', StringType(), True),
            StructField('country', StringType(), True),
            StructField('source_id', StringType(), True),
            StructField('keywords', StringType(), True),
            StructField('sentiment_stats', StringType(), True),
            StructField('source_icon', StringType(), True),
            StructField('image_url', StringType(), True),
            StructField('sentiment', StringType(), True),
            StructField('ai_tag', StringType(), True),
            StructField('link', StringType(), True),
            StructField('creator', StringType(), True),
            StructField('source_priority', StringType(), True),
            StructField('content', StringType(), True),
            StructField('description', StringType(), True),
            StructField('duplicate', StringType(), True),
            StructField('ai_org', StringType(), True),
            StructField('pubDate', StringType(), True),
            StructField('ai_region', StringType(), True),
            StructField('video_url', StringType(), True),
            StructField('language', StringType(), True),
            StructField('category', StringType(), True)
])

free_malaysia_today_schema = StructType([
            StructField('headline', StringType(), True),
            StructField('author', StringType(), True),
            StructField('publish_time', StringType(), True),
            StructField('content', StringType(), True),
])

nst_schema = StructType([
            StructField('title', StringType(), True),
            StructField('author', StringType(), True),
            StructField('published_at', StringType(), True),
            StructField('description', StringType(), True),
])

selangor_journal_schema = StructType([
            StructField('url', StringType(), True),
            StructField('headline', StringType(), True),
            StructField('date of published', StringType(), True),
            StructField('article_content', StringType(), True),
])

the_borneo_post_schema = StructType([
            StructField('URL', StringType(), True),
            StructField('Timestamp', StringType(), True),
            StructField('Title', StringType(), True),
            StructField('Content', StringType(), True),
])

the_star_schema = StructType([
            StructField('Title', StringType(), True),
            StructField('Summary', StringType(), True),
            StructField('Link', StringType(), True),
            StructField('Author', StringType(), True),
            StructField('Published Date', StringType(), True),
            StructField('Published Time', StringType(), True),
])


if redis_client.exists_key('newsdata_io_list'):
    df_newsdata_io = spark.createDataFrame(
        pickle.loads(redis_client.get_value('newsdata_io_list')),
        newsdata_io_schema
    )
else:
    df_newsdata_io = hdfs_client.read_file(file_format = 'csv', source_path = 'newsdata_io.csv', header = True, multiline = True)
    redis_client.set_key_value('newsdata_io_list', pickle.dumps(df_newsdata_io.collect()), seconds = 5 * 60)

if redis_client.exists_key('free_malaysia_today_list'):
    df_free_malaysia_today = spark.createDataFrame(
        pickle.loads(redis_client.get_value('free_malaysia_today_list')),
        free_malaysia_today_schema
    )
else:
    df_free_malaysia_today = hdfs_client.read_file(file_format = 'csv', source_path = 'free_malaysia_today.csv', header = True, multiline = True)
    redis_client.set_key_value('free_malaysia_today_list', pickle.dumps(df_free_malaysia_today.collect()), seconds = 5 * 60)

if redis_client.exists_key('nst_list'):
    df_nst = spark.createDataFrame(
        pickle.loads(redis_client.get_value('nst_list')),
        nst_schema
    )
else:
    df_nst = hdfs_client.read_file(file_format = 'csv', source_path = 'nst.csv', header = True, multiline = True)
    redis_client.set_key_value('nst_list', pickle.dumps(df_nst.collect()), seconds = 5 * 60)

if redis_client.exists_key('selangor_journal_list'):
    df_selangor_journal = spark.createDataFrame(
        pickle.loads(redis_client.get_value('selangor_journal_list')),
        selangor_journal_schema
    )
else:
    df_selangor_journal = hdfs_client.read_file(file_format = 'csv', source_path = 'selangor_journal.csv', header = True, multiline = True)
    redis_client.set_key_value('selangor_journal_list', pickle.dumps(df_selangor_journal.collect()), seconds = 5 * 60)

if redis_client.exists_key('the_borneo_post_list'):
    df_the_borneo_post = spark.createDataFrame(
        pickle.loads(redis_client.get_value('the_borneo_post_list')),
        the_borneo_post_schema
    )
else:
    df_the_borneo_post = hdfs_client.read_file(file_format = 'csv', source_path = 'the_borneo_post.csv', header = True, multiline = True)
    redis_client.set_key_value('the_borneo_post_list', pickle.dumps(df_the_borneo_post.collect()), seconds = 5 * 60)

if redis_client.exists_key('the_star_list'):
    df_the_star = spark.createDataFrame(
        pickle.loads(redis_client.get_value('the_star_list')),
        the_star_schema
    )
else:
    df_the_star = hdfs_client.read_file(file_format = 'csv', source_path = 'the_star.csv', header = True, multiline = True)
    redis_client.set_key_value('the_star_list', pickle.dumps(df_the_star.collect()), seconds = 5 * 60)

[sudo] password for student: 

In [3]:
# Print schema
print("Selangor Journal :", df_selangor_journal)
print("The Borneo Post:", df_the_borneo_post)
print("Free Malaysia Today :", df_free_malaysia_today)
print("NST :", df_nst)
print("The Star :", df_the_star)
print("Newsdata io :", df_newsdata_io)

Selangor Journal : DataFrame[url: string, headline: string, date of published: string, article_content: string]
The Borneo Post: DataFrame[URL: string, Timestamp: string, Title: string, Content: string]
Free Malaysia Today : DataFrame[headline: string, author: string, publish_time: string, content: string]
NST : DataFrame[title: string, author: string, published_at: string, description: string]
The Star : DataFrame[Title: string, Summary: string, Link: string, Author: string, Published Date: string, Published Time: string]
Newsdata io : DataFrame[title: string, article_id: string, source_url: string, country: string, source_id: string, keywords: string, sentiment_stats: string, source_icon: string, image_url: string, sentiment: string, ai_tag: string, link: string, creator: string, source_priority: string, content: string, description: string, duplicate: string, ai_org: string, pubDate: string, ai_region: string, video_url: string, language: string, category: string]


### 2.1.4 Standardising Column Names and Resolve Inconsistent Schema

In [4]:
""" Newsdata io: 
 1) add 'publisher' column and fill in with respective publisher
 2) rename 'title' column to 'headline', 'description' to 'content', 'pubDate' to 'datetime', 'source_url' to 'url'
"""

df_newsdata_io = (
    df_newsdata_io
    .select('title', 'description', 'pubDate', 'source_url')
    .withColumnRenamed('title', 'headline')
    .withColumnRenamed('description', 'content')
    .withColumnRenamed('pubDate', 'datetime')
    .withColumnRenamed('source_url', 'url')
    .withColumnRenamed('creator', 'author')
    .withColumn('author', lit(''))
    .withColumn('publisher', lit('Newsdata io'))
    .select('url', 'headline', 'content', 'datetime', 'author', 'publisher')
)

df_newsdata_io.show()

+--------------------+--------------------+--------------------+-------------------+------+-----------+
|                 url|            headline|             content|           datetime|author|  publisher|
+--------------------+--------------------+--------------------+-------------------+------+-----------+
|https://www.delco...|Upper Darby polic...|Five shots fired;...|2024-07-24 17:11:54|      |Newsdata io|
|https://www.daily...|Judge rules in Ca...|Castle Rock's att...|2024-07-24 16:51:28|      |Newsdata io|
|https://www.times...|Judge rules in Ca...|Castle Rock's att...|2024-07-24 16:51:28|      |Newsdata io|
|https://www.greel...|Judge rules in Ca...|Castle Rock's att...|2024-07-24 16:51:28|      |Newsdata io|
|https://www.delco...|Pa. State Police ...|A Pennsylvania st...|2024-07-24 16:50:13|      |Newsdata io|
|https://www.forbe...|Jannik Sinner—Wor...|Sinner became the...|2024-07-24 16:35:05|      |Newsdata io|
|https://www.centr...|Case against form...|Michael Corson, 5...|

In [5]:
""" Free Malaysia Today: 
 1) add 'url' columns and fill in with null, add 'publisher' column and fill in with respective publisher
 2) rename 'publish_time' column to 'datetime', 'article_content' to 'content'
"""

df_free_malaysia_today = (
    df_free_malaysia_today
    .withColumnRenamed('publish_time', 'datetime')
    .withColumnRenamed('article_content', 'content')
    .withColumn('url', lit(''))
    .withColumn('publisher', lit('Free Malaysia Today'))
    .select('url', 'headline', 'content', 'datetime', 'author', 'publisher')
)

df_free_malaysia_today.show()

+---+--------------------+--------------------+--------------------+-------------+-------------------+
|url|            headline|             content|            datetime|       author|          publisher|
+---+--------------------+--------------------+--------------------+-------------+-------------------+
|   |Cops expand Farah...|PETALING JAYA: Po...|2024-07-20T15:11:...|FMT Reporters|Free Malaysia Today|
|   |Teacher loses RM1...|PETALING JAYA: De...|2024-07-20T07:24:...|FMT Reporters|Free Malaysia Today|
|   |4 teens held on s...|PETALING JAYA: Fo...|2024-07-19T12:15:...|FMT Reporters|Free Malaysia Today|
|   |Kulai MP’s office...|KULAI: The Kulai ...|2024-07-19T09:11:...|      Bernama|Free Malaysia Today|
|   |French cops nab k...|PARIS: French pol...|2024-07-19T07:35:...|      Reuters|Free Malaysia Today|
|   |Cabinet to set up...|KUALA LUMPUR: The...|2024-07-19T05:50:...|      Bernama|Free Malaysia Today|
|   |NGO slams ‘delaye...|PUTRAJAYA: The NG...|2024-07-19T04:23:...|Faisa

In [6]:
""" NST: 
 1) add 'url' column and fill in with null, add 'publisher' column and fill in with respective publisher
 2) rename 'title' column to 'headline', 'published_at' to 'datetime', 'description' to 'content'
"""

df_nst = (
    df_nst
    .withColumnRenamed('title', 'headline')
    .withColumnRenamed('published_at', 'datetime')
    .withColumnRenamed('description', 'content')
    .withColumn('url', lit(''))
    .withColumn('publisher', lit('NST'))
    .select('url', 'headline', 'content', 'datetime', 'author', 'publisher')
)

df_nst.show()

+---+--------------------+--------------------+--------------------+--------------------+---------+
|url|            headline|             content|            datetime|              author|publisher|
+---+--------------------+--------------------+--------------------+--------------------+---------+
|   |Elderly men among...|"GEORGE TOWN: Pol...|July 14, 2024 @ 9...|             Bernama|      NST|
|   |Murder victim fou...|"KOTA BARU: Polic...|July 14, 2024 @ 7...|Sharifah Mahsinah...|      NST|
|   |Indonesian woman ...|AMPANG: Police sa...|July 14, 2024 @ 4...|       Mohamad Al As|      NST|
|   |Syndicate members...|"KUALA LUMPUR: So...|July 14, 2024 @ 3...|Mohd Jamilul Anbi...|      NST|
|   |56 foreigners det...|"KUALA LUMPUR: Fi...|July 14, 2024 @ 2...|      Austin Camoens|      NST|
|   |Panic, prayers as...|"BUTLER: A string...|July 14, 2024 @ 2...|                 Afp|      NST|
|   |Remand extended f...|"KUALA LUMPUR: Th...|July 14, 2024 @ 2...|      Austin Camoens|      NST|


In [7]:
""" Selangor Journal: 
 1) add 'author' column and fill in with null, add 'publisher' column and fill in with respective publisher
 2) rename 'date of published' column to 'datetime', 'article_content' to 'content'
"""

df_selangor_journal = (
    df_selangor_journal
    .withColumnRenamed('date of published', 'datetime')
    .withColumnRenamed('article_content', 'content')
    .withColumn('author', lit(''))
    .withColumn('publisher', lit('Selangor Journal'))
    .select('url', 'headline', 'content', 'datetime', 'author', 'publisher')
)

df_selangor_journal.show()

+--------------------+--------------------+--------------------+--------------------+------+----------------+
|                 url|            headline|             content|            datetime|author|       publisher|
+--------------------+--------------------+--------------------+--------------------+------+----------------+
|https://selangorj...|Five charged with...|PORT DICKSON, Jul...|2024-07-19T20:15:...|      |Selangor Journal|
|https://selangorj...|Businessman acqui...|KUALA LUMPUR, Jul...|2024-07-19T17:30:...|      |Selangor Journal|
|https://selangorj...|Ex-babysitter get...|KUALA LUMPUR, Jul...|2024-07-18T21:18:...|      |Selangor Journal|
|https://selangorj...|Ex-MARii CEO gets...|KUALA LUMPUR, Jul...|2024-07-18T21:12:...|      |Selangor Journal|
|https://selangorj...|Nur Farah Kartini...|PEKAN, July 18 — ...|2024-07-18T19:58:...|      |Selangor Journal|
|https://selangorj...|Police seize RM9 ...|KUALA LUMPUR, Jul...|2024-07-18T19:45:...|      |Selangor Journal|
|https://s

In [8]:
""" The Borneo Post: 
 1) add 'author' column and fill in with null, add 'publisher' column and fill in with respective publisher
 2) rename 'URL' column to 'url', 'Timestamp' to 'datetime', 'Title' to 'headline', 'Content' to 'content'
"""

df_the_borneo_post = (
    df_the_borneo_post
    .withColumnRenamed('URL', 'url')
    .withColumnRenamed('Timestamp', 'datetime')
    .withColumnRenamed('Title', 'headline')
    .withColumnRenamed('Content', 'content')
    .withColumn('author', lit(''))
    .withColumn('publisher', lit('The Borneo Post'))
    .select('url', 'headline', 'content', 'datetime', 'author', 'publisher')
) 

df_the_borneo_post.show()

+--------------------+--------------------+--------------------+--------------------+------+---------------+
|                 url|            headline|             content|            datetime|author|      publisher|
+--------------------+--------------------+--------------------+--------------------+------+---------------+
|https://www.thebo...|Cops looking for ...|The police are cu...|2024-07-14T08:46:...|      |The Borneo Post|
|https://www.thebo...|Sibu cops issue 4...|Police handout ph...|2024-07-13T20:31:...|      |The Borneo Post|
|https://www.thebo...|Fire almost destr...|The fire almost c...|2024-07-13T16:09:...|      |The Borneo Post|
|https://www.thebo...|Engineer in Kuchi...|The engineer in h...|2024-07-12T17:58:...|      |The Borneo Post|
|https://www.thebo...|Miri police arres...|The seized drugs....|2024-07-12T13:56:...|      |The Borneo Post|
|https://www.thebo...|S’wak Customs sei...|Norizan (centre) ...|2024-07-10T13:40:...|      |The Borneo Post|
|https://www.thebo.

In [9]:
""" The Star: 
 1) add 'publisher' column and fill in with respective publisher
 2) rename 'Title' column to 'headline', 'Summary' to 'content', 'Link' to 'url', 'Author' to 'author'
 3) concatenate 'Published Date' and 'Published Time' into 'datetime'
"""

df_the_star = (
    df_the_star
    .withColumnRenamed('Title', 'headline')
    .withColumnRenamed('Summary', 'content')
    .withColumnRenamed('Link', 'url')
    .withColumnRenamed('Author', 'author')
    .withColumn('publisher', lit('The Star'))
    .select('url', 'headline', 'content', concat_ws(' ', df_the_star['Published Date'], df_the_star['Published Time']).alias('datetime'), 'author', 'publisher')
)

df_the_star.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+---------+
|                 url|            headline|             content|            datetime|              author|publisher|
+--------------------+--------------------+--------------------+--------------------+--------------------+---------+
|https://www.thest...|CSI for wildlife ...|Malaysia hosted t...|15 July 2024 07:0...|                 N/A| The Star|
|https://www.thest...|20,000 CCTV camer...|KUALA LUMPUR: Kua...|15 July 2024 05:1...|                 N/A| The Star|
|https://www.thest...|Spike in commerci...|KUALA LUMPUR: The...|13 July 2024 11:1...|      Farik Zolkepli| The Star|
|https://www.thest...|Set-jetting holid...|More internationa...|12 July 2024 11:0...|          Gisele Soo| The Star|
|https://www.thest...|Libraries as a cr...|JOHOR BARU: Libra...|15 July 2024 07:0...|Yee Xiang Yun Xia...| The Star|
|https://www.thest...|Commercial crime ...|KUALA LUMPUR: The...|

### 2.1.5 Merging Data

In [10]:
df_merged = (
   df_selangor_journal
    .unionByName(df_the_borneo_post)
    .unionByName(df_free_malaysia_today)
    .unionByName(df_nst)
    .unionByName(df_newsdata_io)
)

df_merged.show()
# df.count() #to check row number

+--------------------+--------------------+--------------------+--------------------+------+----------------+
|                 url|            headline|             content|            datetime|author|       publisher|
+--------------------+--------------------+--------------------+--------------------+------+----------------+
|https://selangorj...|Five charged with...|PORT DICKSON, Jul...|2024-07-19T20:15:...|      |Selangor Journal|
|https://selangorj...|Businessman acqui...|KUALA LUMPUR, Jul...|2024-07-19T17:30:...|      |Selangor Journal|
|https://selangorj...|Ex-babysitter get...|KUALA LUMPUR, Jul...|2024-07-18T21:18:...|      |Selangor Journal|
|https://selangorj...|Ex-MARii CEO gets...|KUALA LUMPUR, Jul...|2024-07-18T21:12:...|      |Selangor Journal|
|https://selangorj...|Nur Farah Kartini...|PEKAN, July 18 — ...|2024-07-18T19:58:...|      |Selangor Journal|
|https://selangorj...|Police seize RM9 ...|KUALA LUMPUR, Jul...|2024-07-18T19:45:...|      |Selangor Journal|
|https://s

### 2.1.6 Exporting Merged Data to HBase and Redis

In [11]:
# Instantiating HBase Client
hbase_client = HBaseClient(host = 'localhost', port = 9090)

In [12]:
if 'news' in [table.decode('utf-8') for table in hbase_client.get_tables()]:
    hbase_client.delete_table(table_name = 'news', disable = True)
else:
    hbase_client.create_table(table_name = 'news', schema = {'cf1' : dict()})

counter = 0
record_list_with_key = []

# Preparing data to insert into HBase
for data in df_merged.select('*').collect():
    record = (
        'k' + str(counter),
    {
        'cf1:url': data['url'] if data['url'] else '',
        'cf1:headline' : data['headline'] if data['headline'] else '',
        'cf1:datetime' : data['datetime'] if data['datetime'] else '',
        'cf1:content' : data['content']if data['content'] else '',
        'cf1:author' : data['author'] if data['author'] else '',
        'cf1:publisher' : data['publisher']
    })
    record_list_with_key.append(record)
    counter += 1

# Inserting data to HBase
hbase_client.put_keys(table_name = 'news', rows = record_list_with_key)

# Caching data in Redis
redis_client.set_key_value('record_list_with_key', pickle.dumps(record_list_with_key), seconds = 30 * 60)

Table 'news' created.


True

In [13]:
pickle.loads(redis_client.get_value('record_list_with_key'))

[('k0',
  {'cf1:url': 'https://selangorjournal.my/2024/07/five-charged-with-causing-friends-death-in-hotel-swimming-pool/',
   'cf1:headline': 'Five charged with causing friend’s death in hotel swimming pool',
   'cf1:datetime': '2024-07-19T20:15:23+08:00',
   'cf1:content': 'PORT DICKSON, July 19 — Five people were charged in the Magistrate’s Court today with causing the death of their friend in a swimming pool last week. The accused — D Maharaj, 36; B Shanmugaservai, 34; Faezatun Firus Arifin, 42; R Praveena, 31; and S Suvipra, 34 — pleaded not guilty when the charge was read before Magistrate Uthman Abd Ghani. They were jointly accused of causing the death of S. Prawinthan, 38, in the hotel swimming pool in Port Dickson at 2am on July 13. The charge, under Section 304A of the Penal Code, read together with Section 34 of the same act, carries a maximum jail term of two years or a fine, or both upon conviction. Deputy Public Prosecutor Nadia Ezzati Mohd Zainal offered bail of RM8,000 

In [14]:
# check whether data has been successfully inserted
row_key_list = []

for i in range(df_merged.count()):
    row_key_list.append('k'+str(i))

hbase_client.read_keys('news', row_key_list, ['cf1:url'])

[(b'k0',
  {b'cf1:url': b'https://selangorjournal.my/2024/07/five-charged-with-causing-friends-death-in-hotel-swimming-pool/'}),
 (b'k1',
  {b'cf1:url': b'https://selangorjournal.my/2024/07/businessman-acquitted-of-cheating-charges-involving-irb-tax-arrears/'}),
 (b'k2',
  {b'cf1:url': b'https://selangorjournal.my/2024/07/ex-babysitter-gets-fine-community-service-for-endangering-child-causing-death/'}),
 (b'k3',
  {b'cf1:url': b'https://selangorjournal.my/2024/07/ex-marii-ceo-gets-one-year-jail-fine-for-deceiving-board-of-directors/'}),
 (b'k4',
  {b'cf1:url': b'https://selangorjournal.my/2024/07/nur-farah-kartinis-remains-laid-to-rest-in-pekan/'}),
 (b'k5',
  {b'cf1:url': b'https://selangorjournal.my/2024/07/police-seize-rm9-mln-in-drugs-after-cocaine-lab-raid/'}),
 (b'k6',
  {b'cf1:url': b'https://selangorjournal.my/2024/07/moroccan-pleads-guilty-to-assaulting-therapist-at-spa/'}),
 (b'k7',
  {b'cf1:url': b'https://selangorjournal.my/2024/07/suspect-in-poisoned-snacks-case-released-o

In [15]:
redis_client.stop_service()
spark.stop()

[sudo] password for student: 